# 연습문제 4

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd /content/gdrive/MyDrive/Colab Notebooks/ecg

/content/gdrive/MyDrive/Colab Notebooks/ecg


In [ ]:
def drawHistory(hist):
  fig = plt.figure(figsize=(6,4))    # 6x4 크기의 figure 생성
  loss_ax = plt.gca()                 # figure의 기본 축 가져오기
  acc_ax = loss_ax.twinx()            # 축을 1개 더 추가

  # 손실 변화 양상을 그래프로 표시
  loss_ax.plot(hist.history['loss'], 'y', label='train loss')
  loss_ax.plot(hist.history['val_loss'], 'r', label='val_loss')
  loss_ax.set_xlabel('epoch')
  loss_ax.set_ylabel('loss')
  loss_ax.legend(loc='lower left')

  # 정확도 변화 양상을 그래프로 표시
  acc_ax.plot(hist.history['accuracy'], 'b', label='train acc')
  acc_ax.plot(hist.history['val_accuracy'], 'g', label='val acc')
  acc_ax.set_ylabel('accuracy')
  acc_ax.legend(loc='upper left')

  plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
data_no = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ecg/ptbdb_normal.csv')   # 정상 데이터
data_ab = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ecg/ptbdb_abnormal.csv') # 비정상 데이터

data_no = np.array(data_no)
data_ab = np.array(data_ab)

nTrain = 3000 # 학습 데이터
nTest = 1000  # 테스트 데이터

# 3. 배치 크기, 에포크 수, 검증 데이터의 비율

In [ ]:
# 학습/테스트 데이터 생성을 위해 정상/비정상 데이터를 통합
x_train = np.concatenate((data_no[:nTrain,:], data_ab[:nTrain,:]),0)
x_test = np.concatenate((data_no[nTrain:nTrain+nTest,:], data_ab[nTrain:nTrain+nTest,:]),0)

# 학습 데이터(x)의 출력 데이터(y)는 학습 데이터 개수만틈의 1, 테스트 데이터 개수만큼의 0으로 구성
y_train = np.concatenate((np.zeros(nTrain,), np.ones(nTrain,)),0)
y_test = np.concatenate((np.zeros(nTest,), np.ones(nTest,)),0)

from tensorflow.keras.utils import to_categorical
# 입력 데이터는 마지막 축을 1 증가 - 합성곱(CNN) 사용
x_train = np.expand_dims(x_train,-1)
x_test = np.expand_dims(x_test,-1)
# 출력 데이터는 원-핫-인코딩
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras import optimizers

model = Sequential()
model.add(layers.Conv1D(filters=16, kernel_size= 3, input_shape=(x_train.shape[1], 1), activation='relu'))
model.add(layers.Conv1D(filters=16, kernel_size= 3, activation='relu'))
model.add(layers.MaxPooling1D(pool_size = 3,strides=2))
model.add(layers.Conv1D(filters=32, kernel_size= 3, input_shape=(x_train.shape[1], 1), activation='relu'))
model.add(layers.Conv1D(filters=32, kernel_size= 3, activation='relu'))
model.add(layers.MaxPooling1D(pool_size = 3,strides=2))
model.add(layers.LSTM(16))
model.add(layers.Dense(units=2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=0.01), metrics=['accuracy'])

# batch_size

In [ ]:
hist = model.fit(x_train, y_train, epochs=50, batch_size=512, validation_split=0.2)

Epoch 1/50
10/10 [==============================] - 17s 125ms/step - loss: 0.6650 - accuracy: 0.5925 - val_loss: 1.0872 - val_accuracy: 0.0000e+00
Epoch 2/50
10/10 [==============================] - 0s 13ms/step - loss: 0.6218 - accuracy: 0.6535 - val_loss: 0.8437 - val_accuracy: 0.5625
Epoch 3/50
10/10 [==============================] - 0s 11ms/step - loss: 0.5840 - accuracy: 0.7131 - val_loss: 1.0110 - val_accuracy: 0.3092
Epoch 4/50
10/10 [==============================] - 0s 12ms/step - loss: 0.5799 - accuracy: 0.7100 - val_loss: 0.8414 - val_accuracy: 0.5133
Epoch 5/50
10/10 [==============================] - 0s 12ms/step - loss: 0.5667 - accuracy: 0.7200 - val_loss: 0.7301 - val_accuracy: 0.5683
Epoch 6/50
10/10 [==============================] - 0s 11ms/step - loss: 0.5692 - accuracy: 0.7110 - val_loss: 0.9309 - val_accuracy: 0.4167
Epoch 7/50
10/10 [==============================] - 0s 12ms/step - loss: 0.5626 - accuracy: 0.7175 - val_loss: 0.7478 - val_accuracy: 0.5517
Epoch 8

In [ ]:
o = model.predict(x_test)
o = np.argmax(o,1)
y_test = np.argmax(y_test,1)
sum(np.equal(y_test,o))/len(y_test)

63/63 [==============================] - 1s 4ms/step


0.9245

In [ ]:
x_train = np.concatenate((data_no[:nTrain,:], data_ab[:nTrain,:]),0)
x_test = np.concatenate((data_no[nTrain:nTrain+nTest,:], data_ab[nTrain:nTrain+nTest,:]),0)
y_train = np.concatenate((np.zeros(nTrain,), np.ones(nTrain,)),0)
y_test = np.concatenate((np.zeros(nTest,), np.ones(nTest,)),0)

from tensorflow.keras.utils import to_categorical
x_train = np.expand_dims(x_train,-1)
x_test = np.expand_dims(x_test,-1)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras import optimizers

model = Sequential()
model.add(layers.Conv1D(filters=16, kernel_size= 3, input_shape=(x_train.shape[1], 1), activation='relu'))
model.add(layers.Conv1D(filters=16, kernel_size= 3, activation='relu'))
model.add(layers.MaxPooling1D(pool_size = 3,strides=2))
model.add(layers.Conv1D(filters=32, kernel_size= 3, input_shape=(x_train.shape[1], 1), activation='relu'))
model.add(layers.Conv1D(filters=32, kernel_size= 3, activation='relu'))
model.add(layers.MaxPooling1D(pool_size = 3,strides=2))
model.add(layers.LSTM(16))
model.add(layers.Dense(units=2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=0.01), metrics=['accuracy'])

In [ ]:
hist = model.fit(x_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
150/150 [==============================] - 7s 23ms/step - loss: 0.6108 - accuracy: 0.6779 - val_loss: 0.7409 - val_accuracy: 0.5575
Epoch 2/50
150/150 [==============================] - 2s 13ms/step - loss: 0.5842 - accuracy: 0.7104 - val_loss: 0.9068 - val_accuracy: 0.4500
Epoch 3/50
150/150 [==============================] - 2s 12ms/step - loss: 0.5739 - accuracy: 0.7140 - val_loss: 0.7249 - val_accuracy: 0.5925
Epoch 4/50
150/150 [==============================] - 1s 10ms/step - loss: 0.5690 - accuracy: 0.7183 - val_loss: 0.7274 - val_accuracy: 0.5383
Epoch 5/50
150/150 [==============================] - 2s 13ms/step - loss: 0.5617 - accuracy: 0.7175 - val_loss: 0.7698 - val_accuracy: 0.4442
Epoch 6/50
150/150 [==============================] - 2s 11ms/step - loss: 0.5498 - accuracy: 0.7202 - val_loss: 0.7545 - val_accuracy: 0.5058
Epoch 7/50
150/150 [==============================] - 2s 12ms/step - loss: 0.5282 - accuracy: 0.7421 - val_loss: 0.6918 - val_accuracy: 0.5767

In [ ]:
o = model.predict(x_test)
o = np.argmax(o,1)
y_test = np.argmax(y_test,1)
sum(np.equal(y_test,o))/len(y_test)

63/63 [==============================] - 1s 3ms/step


0.896

# epochs

In [ ]:
x_train = np.concatenate((data_no[:nTrain,:], data_ab[:nTrain,:]),0)
x_test = np.concatenate((data_no[nTrain:nTrain+nTest,:], data_ab[nTrain:nTrain+nTest,:]),0)
y_train = np.concatenate((np.zeros(nTrain,), np.ones(nTrain,)),0)
y_test = np.concatenate((np.zeros(nTest,), np.ones(nTest,)),0)

from tensorflow.keras.utils import to_categorical
x_train = np.expand_dims(x_train,-1)
x_test = np.expand_dims(x_test,-1)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras import optimizers

model = Sequential()
model.add(layers.Conv1D(filters=16, kernel_size= 3, input_shape=(x_train.shape[1], 1), activation='relu'))
model.add(layers.Conv1D(filters=16, kernel_size= 3, activation='relu'))
model.add(layers.MaxPooling1D(pool_size = 3,strides=2))
model.add(layers.Conv1D(filters=32, kernel_size= 3, input_shape=(x_train.shape[1], 1), activation='relu'))
model.add(layers.Conv1D(filters=32, kernel_size= 3, activation='relu'))
model.add(layers.MaxPooling1D(pool_size = 3,strides=2))
model.add(layers.LSTM(16))
model.add(layers.Dense(units=2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=0.01), metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=100, batch_size=128, validation_split=0.2)

Epoch 1/100
38/38 [==============================] - 8s 72ms/step - loss: 0.6409 - accuracy: 0.6415 - val_loss: 0.6996 - val_accuracy: 0.6525
Epoch 2/100
38/38 [==============================] - 1s 20ms/step - loss: 0.5902 - accuracy: 0.7060 - val_loss: 0.9384 - val_accuracy: 0.3333
Epoch 3/100
38/38 [==============================] - 1s 16ms/step - loss: 0.5695 - accuracy: 0.7231 - val_loss: 0.8510 - val_accuracy: 0.4475
Epoch 4/100
38/38 [==============================] - 1s 14ms/step - loss: 0.5674 - accuracy: 0.7160 - val_loss: 1.1427 - val_accuracy: 0.1392
Epoch 5/100
38/38 [==============================] - 0s 12ms/step - loss: 0.5739 - accuracy: 0.7150 - val_loss: 0.9169 - val_accuracy: 0.4367
Epoch 6/100
38/38 [==============================] - 1s 17ms/step - loss: 0.5535 - accuracy: 0.7269 - val_loss: 0.7773 - val_accuracy: 0.4900
Epoch 7/100
38/38 [==============================] - 1s 13ms/step - loss: 0.5379 - accuracy: 0.7337 - val_loss: 0.6603 - val_accuracy: 0.5383
Epoch 

In [ ]:
o = model.predict(x_test)
o = np.argmax(o,1)
y_test = np.argmax(y_test,1)
sum(np.equal(y_test,o))/len(y_test)

63/63 [==============================] - 1s 3ms/step


0.954

In [ ]:
x_train = np.concatenate((data_no[:nTrain,:], data_ab[:nTrain,:]),0)
x_test = np.concatenate((data_no[nTrain:nTrain+nTest,:], data_ab[nTrain:nTrain+nTest,:]),0)
y_train = np.concatenate((np.zeros(nTrain,), np.ones(nTrain,)),0)
y_test = np.concatenate((np.zeros(nTest,), np.ones(nTest,)),0)

from tensorflow.keras.utils import to_categorical
x_train = np.expand_dims(x_train,-1)
x_test = np.expand_dims(x_test,-1)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras import optimizers

model = Sequential()
model.add(layers.Conv1D(filters=16, kernel_size= 3, input_shape=(x_train.shape[1], 1), activation='relu'))
model.add(layers.Conv1D(filters=16, kernel_size= 3, activation='relu'))
model.add(layers.MaxPooling1D(pool_size = 3,strides=2))
model.add(layers.Conv1D(filters=32, kernel_size= 3, input_shape=(x_train.shape[1], 1), activation='relu'))
model.add(layers.Conv1D(filters=32, kernel_size= 3, activation='relu'))
model.add(layers.MaxPooling1D(pool_size = 3,strides=2))
model.add(layers.LSTM(16))
model.add(layers.Dense(units=2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=0.01), metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=25, batch_size=128, validation_split=0.2)

Epoch 1/25
38/38 [==============================] - 6s 22ms/step - loss: 0.6109 - accuracy: 0.6844 - val_loss: 0.8562 - val_accuracy: 0.4683
Epoch 2/25
38/38 [==============================] - 0s 8ms/step - loss: 0.5924 - accuracy: 0.6973 - val_loss: 1.0283 - val_accuracy: 0.3425
Epoch 3/25
38/38 [==============================] - 0s 8ms/step - loss: 0.5747 - accuracy: 0.7148 - val_loss: 0.7729 - val_accuracy: 0.5850
Epoch 4/25
38/38 [==============================] - 0s 9ms/step - loss: 0.5745 - accuracy: 0.7190 - val_loss: 0.9408 - val_accuracy: 0.4608
Epoch 5/25
38/38 [==============================] - 0s 10ms/step - loss: 0.5715 - accuracy: 0.7167 - val_loss: 0.7686 - val_accuracy: 0.5492
Epoch 6/25
38/38 [==============================] - 0s 9ms/step - loss: 0.5579 - accuracy: 0.7265 - val_loss: 0.7242 - val_accuracy: 0.5150
Epoch 7/25
38/38 [==============================] - 0s 8ms/step - loss: 0.5511 - accuracy: 0.7237 - val_loss: 0.9025 - val_accuracy: 0.4842
Epoch 8/25
38/38 [

In [ ]:
o = model.predict(x_test)
o = np.argmax(o,1)
y_test = np.argmax(y_test,1)
sum(np.equal(y_test,o))/len(y_test)

63/63 [==============================] - 1s 3ms/step


0.8795

# validation_split

In [ ]:
x_train = np.concatenate((data_no[:nTrain,:], data_ab[:nTrain,:]),0)
x_test = np.concatenate((data_no[nTrain:nTrain+nTest,:], data_ab[nTrain:nTrain+nTest,:]),0)
y_train = np.concatenate((np.zeros(nTrain,), np.ones(nTrain,)),0)
y_test = np.concatenate((np.zeros(nTest,), np.ones(nTest,)),0)

from tensorflow.keras.utils import to_categorical
x_train = np.expand_dims(x_train,-1)
x_test = np.expand_dims(x_test,-1)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras import optimizers

model = Sequential()
model.add(layers.Conv1D(filters=16, kernel_size= 3, input_shape=(x_train.shape[1], 1), activation='relu'))
model.add(layers.Conv1D(filters=16, kernel_size= 3, activation='relu'))
model.add(layers.MaxPooling1D(pool_size = 3,strides=2))
model.add(layers.Conv1D(filters=32, kernel_size= 3, input_shape=(x_train.shape[1], 1), activation='relu'))
model.add(layers.Conv1D(filters=32, kernel_size= 3, activation='relu'))
model.add(layers.MaxPooling1D(pool_size = 3,strides=2))
model.add(layers.LSTM(16))
model.add(layers.Dense(units=2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=0.01), metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=50, batch_size=128, validation_split=0.4)

Epoch 1/50
29/29 [==============================] - 8s 60ms/step - loss: 0.4904 - accuracy: 0.8117 - val_loss: 1.9343 - val_accuracy: 0.0000e+00
Epoch 2/50
29/29 [==============================] - 0s 17ms/step - loss: 0.4535 - accuracy: 0.8333 - val_loss: 1.7937 - val_accuracy: 0.0000e+00
Epoch 3/50
29/29 [==============================] - 1s 18ms/step - loss: 0.4523 - accuracy: 0.8333 - val_loss: 1.7993 - val_accuracy: 0.0000e+00
Epoch 4/50
29/29 [==============================] - 1s 20ms/step - loss: 0.4469 - accuracy: 0.8333 - val_loss: 1.6833 - val_accuracy: 0.0000e+00
Epoch 5/50
29/29 [==============================] - 1s 21ms/step - loss: 0.4084 - accuracy: 0.8353 - val_loss: 1.6039 - val_accuracy: 0.1258
Epoch 6/50
29/29 [==============================] - 1s 20ms/step - loss: 0.4012 - accuracy: 0.8392 - val_loss: 1.7564 - val_accuracy: 0.0512
Epoch 7/50
29/29 [==============================] - 1s 19ms/step - loss: 0.3940 - accuracy: 0.8492 - val_loss: 1.6208 - val_accuracy: 0.06

In [ ]:
o = model.predict(x_test)
o = np.argmax(o,1)
y_test = np.argmax(y_test,1)
sum(np.equal(y_test,o))/len(y_test)

63/63 [==============================] - 1s 3ms/step


0.548

In [ ]:
x_train = np.concatenate((data_no[:nTrain,:], data_ab[:nTrain,:]),0)
x_test = np.concatenate((data_no[nTrain:nTrain+nTest,:], data_ab[nTrain:nTrain+nTest,:]),0)
y_train = np.concatenate((np.zeros(nTrain,), np.ones(nTrain,)),0)
y_test = np.concatenate((np.zeros(nTest,), np.ones(nTest,)),0)

from tensorflow.keras.utils import to_categorical
x_train = np.expand_dims(x_train,-1)
x_test = np.expand_dims(x_test,-1)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras import optimizers

model = Sequential()
model.add(layers.Conv1D(filters=16, kernel_size= 3, input_shape=(x_train.shape[1], 1), activation='relu'))
model.add(layers.Conv1D(filters=16, kernel_size= 3, activation='relu'))
model.add(layers.MaxPooling1D(pool_size = 3,strides=2))
model.add(layers.Conv1D(filters=32, kernel_size= 3, input_shape=(x_train.shape[1], 1), activation='relu'))
model.add(layers.Conv1D(filters=32, kernel_size= 3, activation='relu'))
model.add(layers.MaxPooling1D(pool_size = 3,strides=2))
model.add(layers.LSTM(16))
model.add(layers.Dense(units=2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=0.01), metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=100, batch_size=256, validation_split=0.1)

Epoch 1/100
22/22 [==============================] - 5s 32ms/step - loss: 0.6606 - accuracy: 0.6026 - val_loss: 0.7275 - val_accuracy: 0.5933
Epoch 2/100
22/22 [==============================] - 0s 11ms/step - loss: 0.6039 - accuracy: 0.6935 - val_loss: 0.6143 - val_accuracy: 0.5950
Epoch 3/100
22/22 [==============================] - 0s 10ms/step - loss: 0.5871 - accuracy: 0.7031 - val_loss: 0.8342 - val_accuracy: 0.4067
Epoch 4/100
22/22 [==============================] - 0s 10ms/step - loss: 0.5798 - accuracy: 0.7004 - val_loss: 0.7371 - val_accuracy: 0.4833
Epoch 5/100
22/22 [==============================] - 0s 8ms/step - loss: 0.5840 - accuracy: 0.6957 - val_loss: 0.7906 - val_accuracy: 0.4517
Epoch 6/100
22/22 [==============================] - 0s 9ms/step - loss: 0.5721 - accuracy: 0.7059 - val_loss: 0.8031 - val_accuracy: 0.3950
Epoch 7/100
22/22 [==============================] - 0s 8ms/step - loss: 0.5791 - accuracy: 0.6941 - val_loss: 0.8014 - val_accuracy: 0.4500
Epoch 8/1

In [ ]:
o = model.predict(x_test)
o = np.argmax(o,1)
y_test = np.argmax(y_test,1)
sum(np.equal(y_test,o))/len(y_test)

63/63 [==============================] - 1s 4ms/step


0.9565